In [1]:
// knex

In [2]:
// imports
const moment = require('moment')

In [1]:
// import and initialize

const knex = require('knex')({
  client: 'pg',
  connection: {
    host: '127.0.0.1',
    user: 'postgres',
    password: 'postgres',
    database: 'db1'
  },
  migrations: {
    tableName: 'migrations'
  }
});

// *** ONLY INITIALIZE ONCE ***
// export connection and import in other modules

// module.exports = knex;

In [4]:
// create table

(async (knex) => {
  await knex.schema.createTable('people', (table) => {
    table.increments(); // id (primary key)
    table.string('first_name'); // first name (string)
    table.string('last_name'); // first name (string)
    table.timestamps();
  })
})(knex)

In [5]:
// insert record

// 1
(async (knex) => {
  const personId = await knex('people')
    .insert({
      first_name: 'Kakashi',
      last_name: 'Hatake',
      created_at: moment(),
      updated_at: moment(),
    })
    .returning('id')
  console.log(personId)
})(knex)

[ 1 ]


In [6]:
// insert record 

// 2
(async (knex) => {
  const personId = await knex('people')
    .insert({
      first_name: 'Tenzo',
      last_name: 'Yamato',
      created_at: moment(),
      updated_at: moment(),
    })
    .returning('id')
  console.log(personId)
})(knex)

[ 2 ]


In [7]:
// update

// version 1
  // skipped values must be specified ({ skippedKey: undefined })

(async (knex) => {
  const personId = await knex('people')
    .where({
      id: 1
    })
    .update({
      first_name: 'Tenzo',
      last_name: 'Yamato',
      created_at: undefined, // don't change this value
      updated_at: moment(),
    })
    .returning('id')
  console.log(personId)
})(knex)

[ 1 ]


In [8]:
// update

// version 2
  // get object, update properties, use as input for update command

(async (knex) => {
  // get
  const person = await knex('people')
    .where({ id: 2 })
    .first();
  console.log(person);
  // update properties on user object
  person.first_name = 'Kakashi'
  person.last_name = 'Hatake'
  person.updated_at = moment()
  // update (using updated user object)
  const personId = await knex('people')
    .where({ id: 2 })
    .update(person)
    .returning('id')
  console.log(personId);
})(knex)

{
  id: 2,
  first_name: 'Tenzo',
  last_name: 'Yamato',
  created_at: 2020-04-15T18:48:19.145Z,
  updated_at: 2020-04-15T18:48:19.145Z
}
[ 2 ]


In [9]:
// delete

// return empty array if no match

(async (knex) => {
  const personId = await knex('people')
    .where({ id: 3 })
    .returning('id')
    .del()
  console.log(personId);
})(knex)

[]


In [15]:
// ----------
// select
// ----------

(async (knex) => {
  const result = await knex
    .select(["first_name", "last_name"]) // default to '*' if no argument provided
    .from('people')
    .whereIn('first_name', [
      'Kakashi', 
      'Tenzo'
    ])
    .groupBy('first_name')
    .groupBy('last_name')
    .having(knex.raw('count(*) >= 1'))
  console.log('result', result);
})(knex)

// SELECT first_name, last_name FROM people
// WHERE first_name IN ('Kakashi', 'Tenzo')
// GROUP BY first_name, last_name
// HAVING COUNT(*) >= 1;

result [
  { first_name: 'Kakashi', last_name: 'Hatake' },
  { first_name: 'Tenzo', last_name: 'Yamato' }
]


In [21]:
// select (raw)

(async (knex) => {
  const query = `
  SELECT * FROM people
  WHERE first_name='Kakashi';
  `
  const result = await knex.raw(query);
  const { rows } = result;
  console.log(rows);
})(knex)

[
  {
    id: 2,
    first_name: 'Kakashi',
    last_name: 'Hatake',
    created_at: 2020-04-15T18:48:19.145Z,
    updated_at: 2020-04-15T18:48:19.194Z
  }
]


In [3]:
// distinct

(async(knex) => {
  const distinctFirstNames = await knex('people')
    .distinct('first_name');
  console.log(distinctFirstNames)
})(knex)

// SELECT DISTINCT 'first_name', 'last_name' FROM 'customers'

[ { first_name: 'Kakashi' }, { first_name: 'Tenzo' } ]


In [8]:
// groupby

(async (knex) => {
  const groups = await knex('people')
    .select('first_name', knex.raw('count(*) as people_count'))
    .groupBy('first_name')
  console.log(groups)
})(knex)

[
  { first_name: 'Kakashi', people_count: '1' },
  { first_name: 'Tenzo', people_count: '1' }
]


In [18]:
// min, max, sum, avg
(async (knex) => {
  const max = await knex('people')
    .max('id')
  const min = await knex('people')
    .min('id')
  const sum = await knex('people')
    .sum('id')
  const avg = await knex('people')
    .avg('id')
  const stats = {
    max,
    min,
    sum,
    avg
  };
  console.log(stats)
})(knex)

{
  max: [ { max: 2 } ],
  min: [ { min: 1 } ],
  sum: [ { sum: '3' } ],
  avg: [ { avg: '1.5000000000000000' } ]
}


In [20]:
// union

(async (knex) => {
  const results = await knex('people')
    .select('first_name', 'last_name')
    .where({ first_name: 'Kakashi' })
    .union([
      knex('people')
        .select('first_name', 'last_name')
        .where({ first_name: 'Tenzo' })
    ])
  console.log(results)
})(knex)

[
  { first_name: 'Kakashi', last_name: 'Hatake' },
  { first_name: 'Tenzo', last_name: 'Yamato' }
]


In [ ]:
// join



In [21]:
// orderBy

(async (knex) => {
  results = await knex('people')
    .select('first_name', 'last_name')
    .orderBy([
      { column: 'first_name', order: 'asc' }, // primary criteria
      { column: 'last_name', order: 'desc' } // secondary criteria
    ])
  console.log(results)
})(knex)

[
  { first_name: 'Kakashi', last_name: 'Hatake' },
  { first_name: 'Tenzo', last_name: 'Yamato' }
]
